## Process


### Extraccion de datos


Los cruces fueron extraidos de la pagina oficial:

https://isasurf.org/event/paris-2024/#

In [2]:
import numpy as np
import pandas as pd

dataframe = {'Heat':['HEAT1','HEAT1','HEAT1','HEAT2','HEAT2','HEAT2','HEAT3','HEAT3','HEAT3','HEAT4','HEAT4','HEAT4','HEAT5','HEAT5','HEAT5','HEAT6','HEAT6','HEAT6','HEAT7','HEAT7','HEAT7','HEAT8','HEAT8','HEAT8'],
             'Name':['Ethan Ewing','Tim Elter','Jordy Smith','Joan Duru' ,'Jack Robinson','Matthew McGillivray','Alonso Correa','Filipe Toledo',
                     'Kanoa lgarashi','Gabriel Medina','Connor OLeary','Bryan Perez','Ramzi Boukhiam','Billy Stairmand','João Chianca',
                     'Andy Criere','John John Florence','Alan Cleland','Kauli Vaast','Lucca Mesinas','Griffin Colapinto','Rio Waida','Leonardo Fioravanti',
                     'Reo Inaba']
             }

heats = pd.DataFrame(dataframe)
print(heats)

     Heat                 Name
0   HEAT1          Ethan Ewing
1   HEAT1            Tim Elter
2   HEAT1          Jordy Smith
3   HEAT2            Joan Duru
4   HEAT2        Jack Robinson
5   HEAT2  Matthew McGillivray
6   HEAT3        Alonso Correa
7   HEAT3        Filipe Toledo
8   HEAT3       Kanoa lgarashi
9   HEAT4       Gabriel Medina
10  HEAT4        Connor OLeary
11  HEAT4          Bryan Perez
12  HEAT5       Ramzi Boukhiam
13  HEAT5      Billy Stairmand
14  HEAT5         João Chianca
15  HEAT6          Andy Criere
16  HEAT6   John John Florence
17  HEAT6         Alan Cleland
18  HEAT7          Kauli Vaast
19  HEAT7        Lucca Mesinas
20  HEAT7    Griffin Colapinto
21  HEAT8            Rio Waida
22  HEAT8  Leonardo Fioravanti
23  HEAT8            Reo Inaba


In [3]:
nums = [4133,13157,564,621,3442,3737,1551,1456,3896,1085,2838,10252,1349,2038,9167,2649,199,9341,8990,4391,3165,9112,2656,1490]
heats['url'] = nums
print(heats)

     Heat                 Name    url
0   HEAT1          Ethan Ewing   4133
1   HEAT1            Tim Elter  13157
2   HEAT1          Jordy Smith    564
3   HEAT2            Joan Duru    621
4   HEAT2        Jack Robinson   3442
5   HEAT2  Matthew McGillivray   3737
6   HEAT3        Alonso Correa   1551
7   HEAT3        Filipe Toledo   1456
8   HEAT3       Kanoa lgarashi   3896
9   HEAT4       Gabriel Medina   1085
10  HEAT4        Connor OLeary   2838
11  HEAT4          Bryan Perez  10252
12  HEAT5       Ramzi Boukhiam   1349
13  HEAT5      Billy Stairmand   2038
14  HEAT5         João Chianca   9167
15  HEAT6          Andy Criere   2649
16  HEAT6   John John Florence    199
17  HEAT6         Alan Cleland   9341
18  HEAT7          Kauli Vaast   8990
19  HEAT7        Lucca Mesinas   4391
20  HEAT7    Griffin Colapinto   3165
21  HEAT8            Rio Waida   9112
22  HEAT8  Leonardo Fioravanti   2656
23  HEAT8            Reo Inaba   1490


In [4]:
urls = []

for name in heats.iterrows():
  nombre = name[1].iloc[1]
  if nombre == 'João Chianca':
    nombre = 'Joao Chianca'
  norm_name= nombre.replace(' ', '-').lower()
  url = f'https://www.worldsurfleague.com/athletes/{name[1].iloc[2]}/{norm_name}?isearch=true&scategory=all'
  urls.append(url)
print(urls)


['https://www.worldsurfleague.com/athletes/4133/ethan-ewing?isearch=true&scategory=all', 'https://www.worldsurfleague.com/athletes/13157/tim-elter?isearch=true&scategory=all', 'https://www.worldsurfleague.com/athletes/564/jordy-smith?isearch=true&scategory=all', 'https://www.worldsurfleague.com/athletes/621/joan-duru?isearch=true&scategory=all', 'https://www.worldsurfleague.com/athletes/3442/jack-robinson?isearch=true&scategory=all', 'https://www.worldsurfleague.com/athletes/3737/matthew-mcgillivray?isearch=true&scategory=all', 'https://www.worldsurfleague.com/athletes/1551/alonso-correa?isearch=true&scategory=all', 'https://www.worldsurfleague.com/athletes/1456/filipe-toledo?isearch=true&scategory=all', 'https://www.worldsurfleague.com/athletes/3896/kanoa-lgarashi?isearch=true&scategory=all', 'https://www.worldsurfleague.com/athletes/1085/gabriel-medina?isearch=true&scategory=all', 'https://www.worldsurfleague.com/athletes/2838/connor-oleary?isearch=true&scategory=all', 'https://www.w

#### Obteniendo las estadisticas de cada atleta

In [5]:
import requests
from bs4 import BeautifulSoup

def get_stats (url, name, heat, filas):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, 'html.parser')

  table = soup.find_all('div', class_='new-athlete-stats__body')

  headers = table[1].find_all('th')

  columns =['Heat','Name']
  columns.append(headers[0].find('span').get_text(strip=True))

  columns.append('Average Points per Events')

  rows = table[1].find_all('tr')

  count=0;

  for row in rows:
    if not count:
      count=1
      continue
    celd = [heat, name]
    celdas = row.find_all('td')

    if(celdas[2].get_text(strip=True) == '-'):
      continue

    celd.append(celdas[0].get_text(strip=True))



    points = float(celdas[2].get_text(strip=True).replace(',',''))

    events = float(celdas[3].get_text(strip=True).replace(',',''))

    prom = points/events
    celd.append(prom)

    filas.append(celd)




  return filas, columns


In [7]:
filas = []
columns = []

for i,url in enumerate(urls):

  temp = get_stats(url, heats['Name'][i], heats['Heat'][i], filas)
  columns = temp[1]
  filas = temp[0]



df2 = pd.DataFrame(filas, columns=columns)


print(df2)

      Heat         Name  Year  Average Points per Events
0    HEAT1  Ethan Ewing  2024                3418.571429
1    HEAT1  Ethan Ewing  2023                4370.909091
2    HEAT1  Ethan Ewing  2022                4026.363636
3    HEAT1  Ethan Ewing  2021                2210.000000
4    HEAT1  Ethan Ewing  2017                 977.272727
..     ...          ...   ...                        ...
193  HEAT8    Reo Inaba  2016                 261.428571
194  HEAT8    Reo Inaba  2015                 139.333333
195  HEAT8    Reo Inaba  2014                 125.800000
196  HEAT8    Reo Inaba  2013                 141.400000
197  HEAT8    Reo Inaba  2012                 115.750000

[198 rows x 4 columns]


In [291]:
df2.to_csv('heats.csv', index=False)

### Ronda 1

In [20]:

from sklearn.neighbors import KernelDensity

df_unique = df2.drop_duplicates(subset='Name', keep='first')

kdes=[]


for name in df_unique['Name']:
  points = df2.loc[df2['Name'] == name, 'Average Points per Events']
  dataset = points.values[:, np.newaxis]
  kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(dataset)

  kdes.append([name,kde])


In [21]:
def simulate_tournament(kdes):

  oficial_results=[]

  for i in range(8):
    name_result=[]
    a=i*3

    for _ in range(1000):
      simulations = []
      for j in range(3):
        simulated_points = []
        while True:
          simulated_points = kdes[a+j][1].sample(1).flatten()
          if simulated_points[0] > 0:
            break
        simulations.append((kdes[a+j][0],simulated_points[0]))
     

      simulations.sort(key=lambda x: x[1], reverse=True)

      name_result.append([sim[0] for sim in simulations])


    name_results = pd.DataFrame(name_result, columns=['1st', '2nd', '3rd'])

    top_3_names = name_results.apply(pd.Series.value_counts).fillna(0).reset_index()

    top_3_names.columns = ['Name', '1st', '2nd', '3rd']

    top_3_names = top_3_names.sort_values(by='1st', ascending=False)

    count = 0
    
    oficial_results.append(top_3_names)
    

  return oficial_results





oficial_results = simulate_tournament(kdes)
print(oficial_results)

[          Name    1st    2nd     3rd
1  Jordy Smith  668.0  332.0     0.0
0  Ethan Ewing  332.0  668.0     0.0
2    Tim Elter    0.0    0.0  1000.0,                   Name  1st  2nd  3rd
0        Jack Robinson  587  102  311
2  Matthew McGillivray  263  412  325
1            Joan Duru  150  486  364,              Name    1st  2nd    3rd
1   Filipe Toledo  784.0  158   58.0
2  Kanoa lgarashi  216.0  784    0.0
0   Alonso Correa    0.0   58  942.0,              Name    1st    2nd     3rd
2  Gabriel Medina  912.0   88.0     0.0
1   Connor OLeary   88.0  912.0     0.0
0     Bryan Perez    0.0    0.0  1000.0,               Name  1st  2nd  3rd
1     João Chianca  444  156  400
2   Ramzi Boukhiam  320  342  338
0  Billy Stairmand  236  502  262,                  Name     1st    2nd    3rd
2  John John Florence  1000.0    0.0    0.0
0        Alan Cleland     0.0  663.0  337.0
1         Andy Criere     0.0  337.0  663.0,                 Name  1st  2nd  3rd
0  Griffin Colapinto  923   66   11
2

### Ronda 2

In [28]:
import random
kdes_round_2 = []

for heat in oficial_results:
  count=0
  for name in heat.iterrows():
    if count==0:
      count= 1
      continue
    nombre = name[1].iloc[0]
    for x in kdes:
      if nombre == x[0]:
        kdes_round_2.append(x)
        break
      


random.shuffle(kdes_round_2)

pairs = [(kdes_round_2[i], kdes_round_2[i+1]) for i in range(0, len(kdes_round_2), 2)]

print(pairs)


[(['Alan Cleland', KernelDensity(bandwidth=0.5)], ['Alonso Correa', KernelDensity(bandwidth=0.5)]), (['Bryan Perez', KernelDensity(bandwidth=0.5)], ['Connor OLeary', KernelDensity(bandwidth=0.5)]), (['Leonardo Fioravanti', KernelDensity(bandwidth=0.5)], ['Billy Stairmand', KernelDensity(bandwidth=0.5)]), (['Lucca Mesinas', KernelDensity(bandwidth=0.5)], ['Andy Criere', KernelDensity(bandwidth=0.5)]), (['Joan Duru', KernelDensity(bandwidth=0.5)], ['Ethan Ewing', KernelDensity(bandwidth=0.5)]), (['Kanoa lgarashi', KernelDensity(bandwidth=0.5)], ['Kauli Vaast', KernelDensity(bandwidth=0.5)]), (['Reo Inaba', KernelDensity(bandwidth=0.5)], ['Tim Elter', KernelDensity(bandwidth=0.5)]), (['Matthew McGillivray', KernelDensity(bandwidth=0.5)], ['Ramzi Boukhiam', KernelDensity(bandwidth=0.5)])]


In [31]:
def simulate_round_2(kdes):

  oficial_results=[]

  for i in range(8):
    name_result=[]
    for _ in range(1000):
      simulations = []
      for j in range(2):
        simulated_points = []
        while True:
          simulated_points = kdes[i][j][1].sample(1).flatten()
          if simulated_points[0] > 0:
            break
        simulations.append((kdes[i][j][0],simulated_points[0]))

      simulations.sort(key=lambda x: x[1], reverse=True)
      name_result.append([sim[0] for sim in simulations])

    name_results = pd.DataFrame(name_result, columns=['1st', '2nd'])

    top_by_heat = name_results.apply(pd.Series.value_counts).fillna(0).reset_index()

    top_by_heat.columns = ['Name', '1st', '2nd']

    top_by_heat = top_by_heat.sort_values(by='1st', ascending=False)

    count = 0
    
    oficial_results.append(top_by_heat)

  return oficial_results





round_2_results = simulate_round_2(pairs)
print(round_2_results)

[            Name  1st  2nd
0   Alan Cleland  560  440
1  Alonso Correa  440  560,             Name     1st     2nd
1  Connor OLeary  1000.0     0.0
0    Bryan Perez     0.0  1000.0,                   Name  1st  2nd
1  Leonardo Fioravanti  860  140
0      Billy Stairmand  140  860,             Name  1st  2nd
1  Lucca Mesinas  506  494
0    Andy Criere  494  506,           Name  1st  2nd
0  Ethan Ewing  808  192
1    Joan Duru  192  808,              Name  1st  2nd
0  Kanoa lgarashi  980   20
1     Kauli Vaast   20  980,         Name  1st  2nd
0  Reo Inaba  885  115
1  Tim Elter  115  885,                   Name  1st  2nd
0  Matthew McGillivray  601  399
1       Ramzi Boukhiam  399  601]


### Ronda 3

In [32]:
kdes_round_3 = []

for heat in round_2_results:
  for i in heat.iterrows():
    nombre = i[1].iloc[0]
    for k in kdes_round_2:
      if k[0] == nombre:
        kdes_round_3.append(k)
        break
    break

for heat in oficial_results:
  for i in heat.iterrows():
    nombre = i[1].iloc[0]
    for k in kdes:
      if k[0] == nombre:
        kdes_round_3.append(k)
        break
    break

random.shuffle(kdes_round_3)
pairs = [(kdes_round_3[i], kdes_round_3[i+1]) for i in range(0, len(kdes_round_3), 2)]
print(pairs)


[(['Matthew McGillivray', KernelDensity(bandwidth=0.5)], ['Ethan Ewing', KernelDensity(bandwidth=0.5)]), (['Lucca Mesinas', KernelDensity(bandwidth=0.5)], ['Alan Cleland', KernelDensity(bandwidth=0.5)]), (['Jordy Smith', KernelDensity(bandwidth=0.5)], ['Filipe Toledo', KernelDensity(bandwidth=0.5)]), (['Reo Inaba', KernelDensity(bandwidth=0.5)], ['Kanoa lgarashi', KernelDensity(bandwidth=0.5)]), (['Connor OLeary', KernelDensity(bandwidth=0.5)], ['John John Florence', KernelDensity(bandwidth=0.5)]), (['Rio Waida', KernelDensity(bandwidth=0.5)], ['Gabriel Medina', KernelDensity(bandwidth=0.5)]), (['João Chianca', KernelDensity(bandwidth=0.5)], ['Leonardo Fioravanti', KernelDensity(bandwidth=0.5)]), (['Jack Robinson', KernelDensity(bandwidth=0.5)], ['Griffin Colapinto', KernelDensity(bandwidth=0.5)])]


In [33]:
def simulate_round_3(kdes):

  oficial_results=[]

  for i in range(8):
    name_result=[]

    for _ in range(1000):
      simulations = []
      for j in range(2):
        simulated_points = []
        while True:
          simulated_points = kdes[i][j][1].sample(1).flatten()
          if simulated_points[0] > 0:
            break
        simulations.append((kdes[i][j][0],simulated_points[0]))

      simulations.sort(key=lambda x: x[1], reverse=True)
      name_result.append([sim[0] for sim in simulations])

    name_results = pd.DataFrame(name_result, columns=['1st', '2nd'])

    top_by_heat = name_results.apply(pd.Series.value_counts).fillna(0).reset_index()

    top_by_heat.columns = ['Name', '1st', '2nd']

    top_by_heat = top_by_heat.sort_values(by='1st', ascending=False)

    oficial_results.append(top_by_heat)

  return oficial_results





round_3_results = simulate_round_3(pairs)
print(round_3_results)

[                  Name  1st  2nd
0          Ethan Ewing  768  232
1  Matthew McGillivray  232  768,             Name  1st  2nd
0   Alan Cleland  576  424
1  Lucca Mesinas  424  576,             Name  1st  2nd
0  Filipe Toledo  522  478
1    Jordy Smith  478  522,              Name     1st     2nd
0  Kanoa lgarashi  1000.0     0.0
1       Reo Inaba     0.0  1000.0,                  Name  1st  2nd
1  John John Florence  902   98
0       Connor OLeary   98  902,              Name  1st  2nd
0  Gabriel Medina  926   74
1       Rio Waida   74  926,                   Name  1st  2nd
1  Leonardo Fioravanti  615  385
0         João Chianca  385  615,                 Name  1st  2nd
0  Griffin Colapinto  605  395
1      Jack Robinson  395  605]


### Cuartos de final

In [34]:
kdes_quarter_finals = []

for heat in round_3_results:
  for i in heat.iterrows():
    nombre = i[1].iloc[0]
    for k in kdes_round_3:
      if k[0] == nombre:
        kdes_quarter_finals.append(k)
        break
    break


pairs = [(kdes_quarter_finals[i], kdes_quarter_finals[i+1]) for i in range(0, len(kdes_quarter_finals), 2)]
print(pairs)


[(['Ethan Ewing', KernelDensity(bandwidth=0.5)], ['Alan Cleland', KernelDensity(bandwidth=0.5)]), (['Filipe Toledo', KernelDensity(bandwidth=0.5)], ['Kanoa lgarashi', KernelDensity(bandwidth=0.5)]), (['John John Florence', KernelDensity(bandwidth=0.5)], ['Gabriel Medina', KernelDensity(bandwidth=0.5)]), (['Leonardo Fioravanti', KernelDensity(bandwidth=0.5)], ['Griffin Colapinto', KernelDensity(bandwidth=0.5)])]


In [35]:
def simulate_quarter(kdes):

  oficial_results=[]

  for i in range(4):
    name_result=[]

    for _ in range(1000):
      simulations = []
      for j in range(2):
        simulated_points = []
        while True:
          simulated_points = kdes[i][j][1].sample(1).flatten()
          if simulated_points[0] > 0:
            break
        simulations.append((kdes[i][j][0],simulated_points[0]))

      simulations.sort(key=lambda x: x[1], reverse=True)
      name_result.append([sim[0] for sim in simulations])

    name_results = pd.DataFrame(name_result, columns=['1st', '2nd'])

    top_by_heat = name_results.apply(pd.Series.value_counts).fillna(0).reset_index()

    top_by_heat.columns = ['Name', '1st', '2nd']

    top_by_heat = top_by_heat.sort_values(by='1st', ascending=False)

    oficial_results.append(top_by_heat)

  return oficial_results

quarter_results = simulate_quarter(pairs)
print(quarter_results)

[           Name  1st  2nd
1   Ethan Ewing  954   46
0  Alan Cleland   46  954,              Name  1st  2nd
0   Filipe Toledo  784  216
1  Kanoa lgarashi  216  784,                  Name  1st  2nd
0      Gabriel Medina  586  414
1  John John Florence  414  586,                   Name  1st  2nd
0    Griffin Colapinto  882  118
1  Leonardo Fioravanti  118  882]


### Semifinal

In [36]:
kdes_semi_finals = []

for heat in quarter_results:
  for i in heat.iterrows():
    nombre = i[1].iloc[0]
    for k in kdes_quarter_finals:
      if k[0] == nombre:
        kdes_semi_finals.append(k)
        break
    break

pairs = [(kdes_semi_finals[i], kdes_semi_finals[i+1]) for i in range(0, len(kdes_semi_finals), 2)]
print(pairs)


[(['Ethan Ewing', KernelDensity(bandwidth=0.5)], ['Filipe Toledo', KernelDensity(bandwidth=0.5)]), (['Gabriel Medina', KernelDensity(bandwidth=0.5)], ['Griffin Colapinto', KernelDensity(bandwidth=0.5)])]


In [37]:
def simulate_semis(kdes):

  oficial_results=[]

  for i in range(2):
    name_result=[]

    for _ in range(1000):
      simulations = []
      for j in range(2):
        simulated_points = []
        while True:
          simulated_points = kdes[i][j][1].sample(1).flatten()
          if simulated_points[0] > 0:
            break
        simulations.append((kdes[i][j][0],simulated_points[0]))

      simulations.sort(key=lambda x: x[1], reverse=True)
      name_result.append([sim[0] for sim in simulations])

    name_results = pd.DataFrame(name_result, columns=['1st', '2nd'])

    top_by_heat = name_results.apply(pd.Series.value_counts).fillna(0).reset_index()

    top_by_heat.columns = ['Name', '1st', '2nd']

    top_by_heat = top_by_heat.sort_values(by='1st', ascending=False)

    oficial_results.append(top_by_heat)

  return oficial_results



semis_results = simulate_semis(pairs)
print(semis_results)

[            Name  1st  2nd
1  Filipe Toledo  690  310
0    Ethan Ewing  310  690,                 Name  1st  2nd
0     Gabriel Medina  843  157
1  Griffin Colapinto  157  843]


In [38]:
kdes_final = []

for heat in semis_results:
  for i in heat.iterrows():
    nombre = i[1].iloc[0]
    for k in kdes_semi_finals:
      if k[0] == nombre:
        kdes_final.append(k)
        break
    break


pairs = [(kdes_final[i], kdes_final[i+1]) for i in range(0, len(kdes_final), 2)]
print(pairs)


[(['Filipe Toledo', KernelDensity(bandwidth=0.5)], ['Gabriel Medina', KernelDensity(bandwidth=0.5)])]


In [39]:
kdes_3rd = []

for heat in semis_results:
  count =1
  for i in heat.iterrows():
    if count:
      count=0
      continue
    nombre = i[1].iloc[0]
    for k in kdes_semi_finals:
      if k[0] == nombre:
        kdes_3rd.append(k)
        break
    break




pairs3 = [(kdes_3rd[i], kdes_3rd[i+1]) for i in range(0, len(kdes_3rd), 2)]
print(pairs3)

[(['Ethan Ewing', KernelDensity(bandwidth=0.5)], ['Griffin Colapinto', KernelDensity(bandwidth=0.5)])]


### 3er puesto

In [40]:
def simulate_3rd(kdes):

  oficial_results=[]

  for i in range(1):
    name_result=[]

    for _ in range(1000):
      simulations = []
      for j in range(2):
        simulated_points = []
        while True:
          simulated_points = kdes[i][j][1].sample(1).flatten()
          if simulated_points[0] > 0:
            break
        simulations.append((kdes[i][j][0],simulated_points[0]))

      simulations.sort(key=lambda x: x[1], reverse=True)
      name_result.append([sim[0] for sim in simulations])

    name_results = pd.DataFrame(name_result, columns=['1st', '2nd'])

    top_by_heat = name_results.apply(pd.Series.value_counts).fillna(0).reset_index()

    top_by_heat.columns = ['Name', '1st', '2nd']

    top_by_heat = top_by_heat.sort_values(by='1st', ascending=False)

    oficial_results.append(top_by_heat)

  return oficial_results





final_results = simulate_3rd(pairs3)
print(f'The 3rd place is {final_results[0]['Name'][0]}')

The 3rd place is Ethan Ewing


### Final

In [42]:
def simulate_final(kdes):

  oficial_results=[]

  for i in range(1):
    name_result=[]

    for _ in range(1000):
      simulations = []
      for j in range(2):
        simulated_points = []
        while True:
          simulated_points = kdes[i][j][1].sample(1).flatten()
          if simulated_points[0] > 0:
            break
        simulations.append((kdes[i][j][0],simulated_points[0]))
   

      simulations.sort(key=lambda x: x[1], reverse=True)
      name_result.append([sim[0] for sim in simulations])

    name_results = pd.DataFrame(name_result, columns=['1st', '2nd'])

    top_by_heat = name_results.apply(pd.Series.value_counts).fillna(0).reset_index()

    top_by_heat.columns = ['Name', '1st', '2nd']

    top_by_heat = top_by_heat.sort_values(by='1st', ascending=False)

    oficial_results.append(top_by_heat)

  return oficial_results





final_results = simulate_final(pairs)
print(f'The winner is {final_results[0]['Name'][0]}')
print(f'The 2nd place is {final_results[0]['Name'][1]}')

The winner is Filipe Toledo
The 2nd place is Gabriel Medina
